## summary of processing logs

Download Bio2RDF SPARQL query  Logs

Delete http parameters

count of uniques  

Prefix Addition and preparing logs for parser

parse queries

seperate valid and unvalid queries

normalize vars in parse trees

Extract triples 

Extract s, o, p

# download datasets from LSQ

In [ ]:
from SPARQLWrapper import SPARQLWrapper, CSV

# SPARQL endpoint
endpoint = "https://lsq.data.dice-research.org/sparql"

# List of Bio2RDF datasets
datasets = ["biomedels", "affymetrix", "bioportal", "ctd", "dbsnp",
    "drugbank", "genage", "gendr", "goa", "hgnc", "homologene",
    "irefindex", "kegg", "linkedspl", "mgi", "ncbigene", "omim", "pharmgkb", "sabiork",
    "sgd", "sidr", "taxonomy", "wormbase"]

# Base SPARQL query template without dataset name, LIMIT, and OFFSET
base_query_template = """
PREFIX lsqv: <http://lsq.aksw.org/vocab#> 
PREFIX prov: <http://www.w3.org/ns/prov#>
SELECT Distinct ?text ?timeStamp From <http://lsq.aksw.org/{dataset}>
WHERE {{
?query lsqv:text ?text . 
?query lsqv:hasRemoteExec ?re .
?re prov:atTime ?timeStamp . 
}}
"""

# Initialize SPARQLWrapper with the endpoint
sparql = SPARQLWrapper(endpoint)
sparql.setTimeout(60)  # Set timeout to 60 seconds

# Pagination settings
limit = 10000  # Adjust based on the endpoint's capabilities
offset = 0

# Filename for the CSV file
filename = "data/bio2rdf_23_lsq.csv"

with open(filename, "wb") as file:
    first_chunk = True
    for dataset in datasets:
        more_results = True
        offset = 0
        while more_results:
            # Format the base query with the current dataset
            paginated_query = base_query_template.format(dataset=dataset) + f" LIMIT {limit} OFFSET {offset}"
            
            # Set the query and return format
            sparql.setQuery(paginated_query)
            sparql.setReturnFormat(CSV)
            
            # Execute the query and get the results
            results = sparql.query().convert()

            if first_chunk:
                # Write the first chunk with headers
                file.write(results)
                first_chunk = False
            else:
                # If not the first chunk, find the header's end and skip it
                header_end = results.find(b'\n') + 1
                if header_end > 0 and len(results) > header_end:
                    file.write(results[header_end:])

            print(f"Fetched rows for {dataset} up to offset {offset}. Results length: {len(results)}")

            # Update the offset for the next iteration
            offset += limit

            # Check if the fetched data is significantly smaller than the expected, indicating we may have fetched all available data for this dataset
            if len(results) < limit * 10:  # Adjust based on your average row size
                more_results = False

print(f"Results have been saved to {filename}")


In [2]:
import pandas as pd
df = pd.read_csv('data2/bio2rdf_23_lsq.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()

31950878


,0,1
0,text,timeStamp
1,PREFIX granatum: <http://chem.deri.ie/granatu...,2013-05-13T01:01:36Z
2,PREFIX biopax: <http://www.biopax.org/release...,2013-07-22T16:37:22Z
3,PREFIX biopax: <http://www.biopax.org/release...,2013-07-22T16:37:23Z
4,PREFIX biopax: <http://www.biopax.org/release...,2013-07-23T08:27:12Z


In [3]:
import pandas as pd
df = pd.read_csv('data2/bio2rdf_23_lsq.csv', dtype=str, header=None)

print(len(df))
df.head()

31950878


,0,1
0,text,timeStamp
1,PREFIX granatum: <http://chem.deri.ie/granatu...,2013-05-13T01:01:36Z
2,PREFIX biopax: <http://www.biopax.org/release...,2013-07-22T16:37:22Z
3,PREFIX biopax: <http://www.biopax.org/release...,2013-07-22T16:37:23Z
4,PREFIX biopax: <http://www.biopax.org/release...,2013-07-23T08:27:12Z


In [ ]:
import pandas as pd

# Assuming the CSV file is named 'data.csv' and is located in the current directory
file_path = 'data/bio2rdf_23_lsq.csv'

# Read the CSV file, parse the 'timeStamp' column as datetime
df = pd.read_csv(file_path, parse_dates=['timeStamp'], header=0)

print(df.head())

print(len(df))

# Get the minimum and maximum values from the 'timeStamp' column
min_time = df['timeStamp'].min()
max_time = df['timeStamp'].max()

# Print the minimum and maximum datetime values
print(f"Minimum timestamp: {min_time}")
print(f"Maximum timestamp: {max_time}")



                                                text                 timeStamp
0  PREFIX  granatum: <http://chem.deri.ie/granatu... 2013-05-13 01:01:36+00:00
1  PREFIX  biopax: <http://www.biopax.org/release... 2013-07-22 16:37:22+00:00
2  PREFIX  biopax: <http://www.biopax.org/release... 2013-07-22 16:37:23+00:00
3  PREFIX  biopax: <http://www.biopax.org/release... 2013-07-23 08:27:12+00:00
4  PREFIX  biopax: <http://www.biopax.org/release... 2013-07-22 16:37:22+00:00
31950877
Minimum timestamp: 2013-05-05 10:40:27+00:00
Maximum timestamp: 2014-09-28 23:53:02+00:00

In [1]:
# count of uniques  

import csv

def extract_unique_queries(input_file, output_file):
    try:
        # Initialize a dictionary to store query counts
        query_counts = {}

        with open(input_file, 'r', newline='\n', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)

            # Iterate through the rows and count queries
            for row in reader:
                query = row['text']  # Assuming 'query' is the correct column name
                query_counts[query] = query_counts.get(query, 0) + 1

        # Sort queries and their counts based on the count column in descending order
        sorted_queries = sorted(query_counts.items(), key=lambda item: item[1], reverse=True)

        # Write sorted queries and their counts to the output CSV file
        with open(output_file, 'w', newline='\n', encoding='utf-8') as outfile:
            writer = csv.writer(outfile, lineterminator='\n')
            writer.writerow(['query', 'count'])  # Write header

            for query, count in sorted_queries:
                writer.writerow([query, count])

        print("Unique queries extracted with counts and written to", output_file)

    except FileNotFoundError:
        print("Error: Input CSV file not found.")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'input_file.csv' with the name of your CSV file and 'output_file.csv' with the desired output filename
extract_unique_queries('data2/bio2rdf_23_lsq.csv', 'data2/unique_bio2rdf_lsq_counts.csv')


Unique queries extracted with counts and written to data2/unique_bio2rdf_lsq_counts.csv


In [2]:
import pandas as pd
df = pd.read_csv('data2/unique_bio2rdf_lsq_counts.csv', dtype=str, header=None)

print(len(df))
df.head()

1519794


,0,1
0,query,count
1,DESCRIBE <http://localhost/ping>\n,5502522
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rd...,189746
3,ASK\nWHERE\n { ?s ?p ?o }\n,128131
4,PREFIX drugbank: <http://www4.wiwiss.fu-berli...,27707


In [3]:
#Prefix Addition 

import pandas as pd

def add_prefix(query):
    # The common prefix to add to each query
    prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX schema: <http://schema.org/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

"""
    return prefix + query

def add_prefix_to_csv(input_file, output_file):
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file)

    # Add the prefix to the "query" column using the add_prefix function
    df['query'] = df['query'].apply(add_prefix)

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_csv_file = 'data2/unique_bio2rdf_lsq_counts.csv' # Replace with the path to your input CSV file
    output_csv_file = "data2/prefixes_added_bio2rdf.csv"  # Replace with the path to your output CSV file
    add_prefix_to_csv(input_csv_file, output_csv_file)
    print('Done!')

Done!


In [4]:
df = pd.read_csv('data2/prefixes_added_bio2rdf.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
# 1051680

1519794


,0,1
0,query,count
1,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,5502522
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,189746
3,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,128131
4,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,27707


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('data2/prefixes_added_bio2rdf.csv')

# Function to apply transformations to the first column
def process_string(input_string):
    # Replace <> with ?class
    processed_string = input_string.replace('<>', '?class')
    
    # Replace http%3A with http:
    processed_string = processed_string.replace('http%3A', 'http:')
    
    # Replace ?rOFFSET with ?r OFFSET
    processed_string = processed_string.replace('?rOFFSET', '?r OFFSET')
    
    # Replace >PREFIX with >\nPREFIX
    processed_string = processed_string.replace('>PREFIX', '>\nPREFIX')
    
    # Replace >SELECT with >\nSELECT
    processed_string = processed_string.replace('>SELECT', '>\nSELECT')
    
    return processed_string

# Apply the function to the first column
df.iloc[:, 0] = df.iloc[:, 0].apply(process_string)

# Write the modified dataframe to a new CSV file
df.to_csv('data2/processed_prefixes_added_bio2rdf.csv', index=False)

print("Processing complete. Output saved to 'processed_prefixes_added_bio2rdf.csv'")


In [ ]:
#  parse queries
# Install Node.js dependencies
!npm install sparqljs csv-parser csv-stringify

# Create the JavaScript file
js_code = """
const fs = require('fs');
const SparqlParser = require('sparqljs').Parser;
const csvParser = require('csv-parser');
const { stringify } = require('csv-stringify');

const parser = new SparqlParser();

async function executeQuery(query) {
  try {
    const parsedQuery = parser.parse(query);
    return JSON.stringify(parsedQuery);
  } catch (error) {
    console.error('Error parsing query:', query);
    return 'Error parsing the query.';
  }
}

async function writeBatchToCsv(batch, outputFile) {
  return new Promise((resolve, reject) => {
    const writeStream = fs.createWriteStream(outputFile, { flags: 'a' });
    const csvStringifier = stringify({ header: false, columns: [ 'Parsed_Query','Count'], delimiter: ',' });

    writeStream.on('error', (error) => {
      reject(error);
    });

    csvStringifier.pipe(writeStream);

    csvStringifier.on('end', () => {
      writeStream.end();
      resolve();
    });

    batch.forEach((entry) => {
      csvStringifier.write([ entry.Parsed_Query , entry.Count ]);
    });

    csvStringifier.end();
  });
}

async function main() {
  const inputCsvFile = 'query_prefixes_added.csv';
  const outputCsvFile = 'wikidata-robotic-parsed.csv';
  const batchSize = 1000000;

  const readStream = fs.createReadStream(inputCsvFile).pipe(csvParser());

  let batch = [];
  for await (const row of readStream) {
    const sparqlQuery = row['query'];
    const count = row['count'];

    const parsedQuery = await executeQuery(sparqlQuery);

    batch.push({ Parsed_Query: parsedQuery ,Count: count });

    if (batch.length >= batchSize) {
      await writeBatchToCsv(batch, outputCsvFile);
      console.log(`Processed ${batch.length} queries.`);
      batch = [];
    }
  }

  if (batch.length > 0) {
    await writeBatchToCsv(batch, outputCsvFile);
  }

  console.log('All SPARQL queries executed and results written to output CSV file.');
}

main();
"""

# Write JavaScript code to a file
with open("optimized.js", "w") as f:
    f.write(js_code)

# Python code to run the JavaScript script
python_code = """
import subprocess

def run_script():
    # Run the JavaScript code using Node.js with an increased memory limit
    result = subprocess.run(['node', '--max-old-space-size=30720', 'optimized.js'], stdout=subprocess.PIPE, text=True)

    # Print the output (JSON representation of the parsed SPARQL query)
    print(result.stdout)
    print('hi')

if __name__ == "__main__":
    run_script()
"""

# Write Python code to a file
with open("optimized_from_js_.py", "w") as f:
    f.write(python_code)

# Execute the script directly
print("Running the script...")

!python optimized_from_js_.py > parseoutput.txt 2>&1

# Print the log output
with open("parseoutput.txt", "r") as log_file:
    output = log_file.read()
    print(output)


In [6]:
# seperate valid queries and unvalid queries

import csv

# Define the input and output file paths
input_file_path = 'data2/parsed.csv'
output_file_path = 'data2/valid_parsed.csv'

# Open the input CSV file for reading
with open(input_file_path, 'r', newline='\n', encoding='utf-8') as input_file:
    # Create a CSV reader without header
    reader = csv.reader(input_file)
    
    # Create a list to store valid rows
    valid_rows = []
    
    for row in reader:
        # Check if the second column is not equal to 'Error parsing the query'
        if len(row) >= 3 and row[2] != 'Error parsing the query.':
            valid_rows.append(row)

# Open the output CSV file for writing
with open(output_file_path, 'w', newline='\n', encoding='utf-8') as output_file:
    # Create a CSV writer with column names
    writer = csv.writer(output_file)
    
    # Write column names
    writer.writerow(['query', 'count' ,'parsed_query'])
    
    # Write the valid rows
    writer.writerows(valid_rows)

print(f"Valid rows have been written to '{output_file_path}'.")


Valid rows have been written to 'data2/valid_parsed.csv'.


In [7]:
import pandas as pd
df = pd.read_csv('data2/valid_parsed.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
#634784

1514545


,0,1,2
0,query,count,parsed_query\r
1,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,5502522,"{""queryType"":""DESCRIBE"",""variables"":[{""termTyp..."
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,189746,"{""queryType"":""SELECT"",""variables"":[{}],""where""..."
3,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,128131,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri..."
4,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,27707,"{""queryType"":""SELECT"",""variables"":[{}],""where""..."


In [1]:
#normalize parse tree

import pandas as pd
import json

# The normalization function from the previous discussion
def find_and_normalize_variables(parsed_query):
    unique_vars = set()

    def find_variables(node):
        if isinstance(node, dict):
            for key, value in node.items():
                if isinstance(value, dict) or isinstance(value, list):
                    find_variables(value)
                elif key == "value" and node.get("termType") == "Variable":
                    unique_vars.add(value)
        elif isinstance(node, list):
            for item in node:
                find_variables(item)

    def normalize_variables(node, var_mapping):
        if isinstance(node, dict):
            for key, value in node.items():
                if isinstance(value, dict) or isinstance(value, list):
                    normalize_variables(value, var_mapping)
                elif key == "value" and node.get("termType") == "Variable":
                    node[key] = var_mapping.get(value, value)
        elif isinstance(node, list):
            for item in node:
                normalize_variables(item, var_mapping)

    find_variables(parsed_query)
    var_mapping = {var: f"var{index+1}" for index, var in enumerate(sorted(unique_vars))}
    normalize_variables(parsed_query, var_mapping)

    if "variables" in parsed_query:
        for variable in parsed_query["variables"]:
            var_name = variable.get("value")
            if var_name in var_mapping:
                variable["value"] = var_mapping[var_name]

    return parsed_query

# Function to apply normalization to each row's parsed query
def process_row(row):
    parsed_query = json.loads(row['parsed_query'])
    normalized_query = find_and_normalize_variables(parsed_query)
    return json.dumps(normalized_query)

# Read the CSV file
df = pd.read_csv('data2/valid_parsed.csv')  # Make sure to replace 'input.csv' with your actual file path

# Normalize parsed query for each row
df['normalized_parse_tree'] = df.apply(process_row, axis=1)

# Write the result to a new CSV file
df.to_csv('data2/tree_normalized.csv', index=False)
print('done')

done


In [2]:
import pandas as pd
df = pd.read_csv('data2/tree_normalized.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
#634784

1514545


,0,1,2,3
0,query,count,parsed_query,normalized_parse_tree
1,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,5502522,"{""queryType"":""DESCRIBE"",""variables"":[{""termTyp...","{""queryType"": ""DESCRIBE"", ""variables"": [{""term..."
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,189746,"{""queryType"":""SELECT"",""variables"":[{}],""where""...","{""queryType"": ""SELECT"", ""variables"": [{}], ""wh..."
3,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,128131,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri...","{""queryType"": ""ASK"", ""where"": [{""type"": ""bgp"",..."
4,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,27707,"{""queryType"":""SELECT"",""variables"":[{}],""where""...","{""queryType"": ""SELECT"", ""variables"": [{}], ""wh..."


In [1]:
import pandas as pd

# Path to your CSV file
file_path = 'data2/tree_normalized.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, lineterminator='\n')

# Count the unique values in the 'normalized_parse_tree' column
unique_values_count = df['normalized_parse_tree'].nunique()

print(f"Number of unique values in 'normalized_parse_tree': {unique_values_count}")


Number of unique values in 'normalized_parse_tree': 1486540


In [1]:
import csv
import json


# Load your CSV data from the input file
input_csv_file = 'data2/tree_normalized.csv'
output_csv_file = 'data2/query_features.csv'
# Define a recursive function to handle nested path types
def extract_predicate_values(items):
    predicates = []
    for item in items:
        if "value" in item:  # Direct predicate
            predicates.append(item["value"])
        elif "pathType" in item:  # Nested path
            # Recursively extract nested predicates
            nested_predicates = extract_predicate_values(item.get("items", []))
            predicates.extend(nested_predicates)
        else:
            predicates.append("Unknown")
    return predicates

# Modify the extract_triples function to use the recursive function
def extract_triples(parse_tree):
    local_triples = []
    if "triples" in parse_tree:
        for triple in parse_tree["triples"]:
            try:
                subject = triple.get("subject", {}).get("value", "Unknown")
                
                # Check if the predicate is a direct value or a path
                if "value" in triple.get("predicate", {}):  # Direct predicate
                    predicate = triple["predicate"]["value"]
                elif "pathType" in triple.get("predicate", {}):  # Path type predicate
                    # Use the recursive function to handle nested path types
                    items = triple["predicate"].get("items", [])
                    predicate_values = extract_predicate_values(items)
                    predicate = ", ".join(predicate_values)
                else:
                    predicate = "nothing"
                
                obj = triple.get("object", {}).get("value", "Unknown")
                local_triples.append((subject, predicate, obj))
            except KeyError as e:
                print(f"Error extracting triple: {e}")
                print("Offending triple:", triple)
    
    # Recursively call the function on child nodes
    for key, value in parse_tree.items():
        if isinstance(value, dict):
            local_triples += extract_triples(value)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    local_triples += extract_triples(item)
    
    return local_triples

# Prepare to read the CSV file and process each row
with open(input_csv_file, 'r') as csvfile, open(output_csv_file, 'w', newline='') as outfile:
    reader = csv.reader(csvfile)
    writer = None  # We'll define this after we read the header row

    for row_index, row in enumerate(reader):
        if row_index == 0:
            # Initialize the CSV writer with the header row from the input file plus the 'triples' column
            fieldnames = row + ['triples']
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            continue

        # Initialize a dictionary for the current row, preserving all input columns
        row_dict = {fieldnames[i]: row[i] for i in range(len(row))}

        # Extract triples from the fourth column if it exists and is not empty
        if len(row) > 3 and row[3]:
            try:
                parse_tree = json.loads(row[3])
                triples = extract_triples(parse_tree)
                # Convert the list of triples to a string format to store in the CSV
                triples_str = ' \\+ '.join([f"{s}, {p}, {o}" for s, p, o in triples])
                row_dict['triples'] = triples_str
            except json.JSONDecodeError:
                row_dict['triples'] = "Invalid JSON"

        # Write the modified row with the triples to the output CSV
        writer.writerow(row_dict)

print("Processing complete. Output written to", output_csv_file)



Processing complete. Output written to data2/query_features.csv


In [2]:
import pandas as pd
df = pd.read_csv('data2/query_features.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
# 1514545

1514545


,0,1,2,3,4
0,query,count,parsed_query,normalized_parse_tree,triples\r
1,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,5502522,"{""queryType"":""DESCRIBE"",""variables"":[{""termTyp...","{""queryType"": ""DESCRIBE"", ""variables"": [{""term...",\r
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,189746,"{""queryType"":""SELECT"",""variables"":[{}],""where""...","{""queryType"": ""SELECT"", ""variables"": [{}], ""wh...","var1, http://www.w3.org/1999/02/22-rdf-syntax-..."
3,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,128131,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri...","{""queryType"": ""ASK"", ""where"": [{""type"": ""bgp"",...","var3, var2, var1\r"
4,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,27707,"{""queryType"":""SELECT"",""variables"":[{}],""where""...","{""queryType"": ""SELECT"", ""variables"": [{}], ""wh...","var1, http://www.w3.org/1999/02/22-rdf-syntax-..."


In [2]:
#find what are the blank nodes symbols

import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data2/query_features.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Counter for cases found
cases_found = 0

# Iterate through the DataFrame
for index, row in df.iterrows():
    # Check if the third column (index 2) starts with 'e_'
    if 'e_' in str(row[2]):
        # Print the entire row
        print(row)
        # Increment the counter
        cases_found += 1
        # If 10 cases have been found, stop
        if cases_found == 10:
            break

if cases_found == 0:
    print("No cases found.")

#e_b
#g_

query                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...
count                                                                 5900
parsed_query             {"queryType":"SELECT","variables":[{}],"where"...
normalized_parse_tree    {"queryType": "SELECT", "variables": [{}], "wh...
triples                  var1, http://bio2rdf.org/homologene_vocabulary...
Name: 218, dtype: object
query                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...
count                                                                 1890
parsed_query             {"queryType":"SELECT","variables":[{}],"where"...
normalized_parse_tree    {"queryType": "SELECT", "variables": [{}], "wh...
triples                  var2, http://bio2rdf.org/ctd_vocabulary:diseas...
Name: 1159, dtype: object
query                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...
count                                                                 1871
parsed_query             {"queryType":"CONSTRUCT"

In [3]:
# count and triples 


import pandas as pd

# Define the path to the input CSV file
input_csv_path = 'data2/query_features.csv'  # Make sure to replace this with the actual path to your CSV file

# Define the path to the output CSV file
output_csv_path = 'data2/count_triples.csv'

# Read the input CSV file into a pandas DataFrame
df = pd.read_csv(input_csv_path)

# Select the 'count' and 'triples' columns
selected_columns = df[['count', 'triples']]

# Write the selected columns to the output CSV file
selected_columns.to_csv(output_csv_path, index=False)

print('CSV file "count_triples.csv" has been created successfully.')


CSV file "count_triples.csv" has been created successfully.


In [4]:
import pandas as pd
df = pd.read_csv('data2/count_triples.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()

1514545


,0,1
0,count,triples
1,5502522,NaN
2,189746,"var1, http://www.w3.org/1999/02/22-rdf-syntax-..."
3,128131,"var3, var2, var1"
4,27707,"var1, http://www.w3.org/1999/02/22-rdf-syntax-..."


In [ ]:
# we have 659 null
import csv

def process_triples(triples):
    entities = []
    predicates = []
    for triple in triples.split(' \\+ '):
        parts = triple.split(', ')
        if len(parts) == 3:
            subj, pred, obj = parts
            if not (subj.startswith('var') or subj.startswith('e_b') or subj.startswith('g_') or 'nonsensical' in subj):
                entities.append(subj)
            if not (obj.startswith('var') or obj.startswith('e_b') or obj.startswith('g_') or 'nonsensical' in obj):
                entities.append(obj)
            if not (pred.startswith('var') or pred.startswith('e_b') or pred.startswith('g_') or 'nonsensical' in pred):
                 predicates.append(pred)
        elif len(parts) > 3:
                entities.append(parts[0])
                entities.append(parts[-1])
                predicates.extend(parts[1:-1])
        else:  # Handling cases where len(parts) == 2
                # entities.extend(parts[:2])
                # predicates.append(parts[-1])
                print("Error: Triple does not have three elements:", triple)
                continue

    return entities, predicates

def write_output(input_csv, output_csv):
    with open(input_csv, mode='r', encoding='utf-8') as infile, \
         open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['entities', 'predicates']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in reader:
            entities, predicates = process_triples(row['triples'])
            row['entities'] = '; '.join(entities)
            row['predicates'] = '; '.join(predicates)
            writer.writerow(row)

# Replace 'input.csv' and 'output.csv' with the actual filenames
input_csv = 'data2/count_triples.csv'
output_csv = 'data2/entity_predicates.csv'

write_output(input_csv, output_csv)
print("Enhanced data with entities and predicates have been saved to", output_csv)


In [6]:
import pandas as pd
df = pd.read_csv('data2/entity_predicates.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
# 634784

1514545


,0,1,2,3
0,count,triples,entities,predicates\r
1,5502522,NaN,NaN,\r
2,189746,"var1, http://www.w3.org/1999/02/22-rdf-syntax-...",http://chem.deri.ie/granatum/chemSpiderURI,http://www.w3.org/1999/02/22-rdf-syntax-ns#type\r
3,128131,"var3, var2, var1",NaN,\r
4,27707,"var1, http://www.w3.org/1999/02/22-rdf-syntax-...",http://www4.wiwiss.fu-berlin.de/drugbank/resou...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type\r


# Schema predicates unique




In [28]:
import pandas as pd

# Read the input CSV file
input_csv = "data2/entity_predicates.csv"  # Update this path to your input CSV file
df = pd.read_csv(input_csv)

# Initialize a dictionary to hold the aggregated counts
aggregated_counts = {}

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Check if the "entities" column value is a string
    if isinstance(row['predicates'], str):
        # Split the entities and convert to a set of unique entities
        unique_entities = set(row['predicates'].strip().split(';'))
    else:
        # If not a string, proceed with an empty set for this row
        unique_entities = set()

    # Aggregate counts across all rows
    for entity in unique_entities:
        entity = entity.strip()
        if entity in aggregated_counts:
            aggregated_counts[entity] += 1
        else:
            aggregated_counts[entity] = 1

# Convert the aggregated dictionary to a DataFrame for writing to CSV
output_df = pd.DataFrame(list(aggregated_counts.items()), columns=['predicate', 'TotalCount'])

# Write the output DataFrame to a CSV file
output_csv = "gold/8_unique_count_predicates_no_repeat.csv"  # Update this path to your output CSV file
output_df.to_csv(output_csv, index=False)

print(f"Aggregated counts written to {output_csv}")


Aggregated counts written to gold/8_unique_count_predicates_no_repeat.csv


In [29]:
import pandas as pd
df = pd.read_csv('gold/8_unique_count_predicates_no_repeat.csv', dtype=str)

print(len(df))
df.head()
#1911

2280


,predicate,TotalCount
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,783992
1,http://bio2rdf.org/goa_vocabulary:process,4731
2,http://www.w3.org/2000/01/rdf-schema#subClassOf,38062
3,http://purl.org/linked-data/cube#codeList,2
4,http://www.w3.org/2004/02/skos/core#prefLabel,15


In [31]:
# extract valid predicates that exist in schema predicate

import csv

# File paths
input_file_path_1 = 'gold/8_unique_count_predicates_no_repeat.csv'  # First input file with two columns: type, TotalCount
input_file_path_2 = 'opredicate.csv'  # Second input file with one column
output_file_path = 'fire/no-resource_heatmap_unique_predicates_no_repeat.csv'   # Output file

# Read the types from the second input file into a set for faster search
types_in_second_file = set()
with open(input_file_path_2, mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip header if there is one
    for row in reader:
        types_in_second_file.add(row[0])

# Read the first input file and write relevant rows to the output file
with open(input_file_path_1, mode='r', newline='') as infile, \
     open(output_file_path, mode='w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    header = next(reader)  # Assuming the first row is a header
    writer.writerow(header)  # Write the header to the output file
    
    for row in reader:
        if row[0] in types_in_second_file:  # Check if the type is in the second file
            writer.writerow(row)  # Write the whole row to the output file
print("Aggregated counts written to heatmap_predicates77.csv")


Aggregated counts written to heatmap_predicates77.csv


In [32]:
import pandas as pd
df = pd.read_csv('fire/no-resource_heatmap_unique_predicates_no_repeat.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()

91


,0,1
0,predicate,TotalCount\r
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,783992\r
2,http://www.w3.org/2000/01/rdf-schema#subClassOf,38062\r
3,http://bio2rdf.org/drugbank_vocabulary:target,47415\r
4,http://bio2rdf.org/drugbank_vocabulary:indication,30381\r


# Unique queries
## Schema classes

In [28]:
# unique count
# extract enties and their count
# ignore query count

import pandas as pd

# Read the input CSV file
input_csv = "data/entity_predicates.csv"  # Update this path to your input CSV file
df = pd.read_csv(input_csv)

# Initialize a dictionary to hold the aggregated counts
aggregated_counts = {}

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Check if the "entities" column value is a string
    if isinstance(row['entities'], str):
        # Split the entities and count occurrences
        entities = row['entities'].strip().split(';')
        row_entities_count = {entity: entities.count(entity) for entity in set(entities)}
    else:
        # If not a string, proceed with an empty dictionary for this row
        row_entities_count = {}
    
    # Multiply each entity's count in the row by the "count" value for the row
    multiplied_counts = {entity: count  for entity, count in row_entities_count.items()}
    
    # Aggregate counts across all rows
    for entity, count in multiplied_counts.items():
        entity = entity.strip()
        if entity in aggregated_counts:
            aggregated_counts[entity] += count
        else:
            aggregated_counts[entity] = count

# Convert the aggregated dictionary to a DataFrame for writing to CSV
output_df = pd.DataFrame(list(aggregated_counts.items()), columns=['Entity', 'TotalCount'])

# Write the output DataFrame to a CSV file
output_csv = "data/8_unique_count_entity.csv"  # Update this path to your output CSV file
output_df.to_csv(output_csv, index=False)

print(f"Aggregated counts written to {output_csv}")


Aggregated counts written to data/8_unique_count_entity.csv


In [29]:
import pandas as pd
df = pd.read_csv('data/8_unique_count_entity.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()
#507

31004


,0,1
0,Entity,TotalCount
1,http://example.org/thing,5
2,http://example.org/string,1
3,false,2
4,12,1


In [ ]:
# extract types and their count

import pandas as pd

# Read the input CSV file
input_csv = "data/8_unique_count_entity.csv"  # Update this path to your input CSV file
df = pd.read_csv(input_csv)

# Initialize a dictionary to hold the aggregated counts
aggregated_counts = {}

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Check if the "entities" column value is a string
    if isinstance(row['types'], str):
        # Split the entities and count occurrences
        entities = row['types'].strip().split('\n')
        row_entities_count = {entity: entities.count(entity) for entity in set(entities)}
    else:
        # If not a string, proceed with an empty dictionary for this row
        row_entities_count = {}
    
    # Multiply each entity's count in the row by the "count" value for the row
    multiplied_counts = {entity: count * row['TotalCount'] for entity, count in row_entities_count.items()}
    
    # Aggregate counts across all rows
    for entity, count in multiplied_counts.items():
        entity = entity.strip()
        if entity in aggregated_counts:
            aggregated_counts[entity] += count
        else:
            aggregated_counts[entity] = count

# Convert the aggregated dictionary to a DataFrame for writing to CSV
output_df = pd.DataFrame(list(aggregated_counts.items()), columns=['type', 'TotalCount'])

# Write the output DataFrame to a CSV file
output_csv = "data/8_unique_count_types.csv"  # Update this path to your output CSV file
output_df.to_csv(output_csv, index=False)

print(f"Aggregated counts written to {output_csv}")


In [ ]:
import pandas as pd
df = pd.read_csv('data/8_unique_count_types3.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()

In [1]:
# extract the types from queries that exist in KG schema

import csv

# File paths
input_file_path_1 = 'data/8_unique_count_entity.csv'  # First input file with two columns: type, TotalCount
input_file_path_2 = 'data/schema_classes_25_March.csv'  # Second input file with one column
output_file_path = 'data/unique_heatmap_types3.csv'   # Output file

# Read the types from the second input file into a set for faster search
types_in_second_file = set()
with open(input_file_path_2, mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip header if there is one
    for row in reader:
        types_in_second_file.add(row[0])

# Read the first input file and write relevant rows to the output file
with open(input_file_path_1, mode='r', newline='') as infile, \
     open(output_file_path, mode='w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    header = next(reader)  # Assuming the first row is a header
    writer.writerow(header)  # Write the header to the output file
    
    for row in reader:
        if row[0] in types_in_second_file:  # Check if the type is in the second file
            writer.writerow(row)  # Write the whole row to the output file
        # else:
        #     print( row[0])
print("Aggregated counts written to unique_heatmap_types.csv")

Aggregated counts written to unique_heatmap_types.csv


In [2]:
import pandas as pd
df = pd.read_csv('data/unique_heatmap_types3.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head()

526


,0,1
0,Entity,TotalCount\r
1,http://bio2rdf.org/drugbank_vocabulary:Drug,121164\r
2,http://bio2rdf.org/hgnc_vocabulary:Resource,198\r
3,http://bio2rdf.org/omim_vocabulary:Gene,69374\r
4,http://bio2rdf.org/hgnc_vocabulary:Gene-Symbol,104\r


In [14]:
# excluded_resources from schema classes

import pandas as pd

# Load the CSV file
df = pd.read_csv('data/unique_heatmap_types3.csv')

# Filter rows where 'types' column does not end with ':Resource'
filtered_df = df[~df['Entity'].str.endswith(':Resource')]

# Save the filtered data to a new CSV file
filtered_df.to_csv('data/excluded_resources_heatmap_types3.csv', index=False)

print('File saved as excluded_resources_heatmap_types.csv with rows not ending in ":Resource".')


File saved as excluded_resources_heatmap_types.csv with rows not ending in ":Resource".
